<b>Initialization</b>

In [2]:
from __future__  import print_function, division

import os
import pandas as pd
import matplotlib.pylab as plt
import scipy.stats as scipy
%matplotlib inline

In [3]:
os.getenv('PUIDATA')

In [9]:
# Federica's function, fixed bug with file type
def getCitiBikeCSV(datestring):
    print ("Downloading", datestring)
    ### First I will heck that it is not already there
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        if os.path.isfile(datestring + "-citibike-tripdata.csv"):
            # if in the current dir just move it
            if os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA")):
                print ("Error moving file!, Please check!")
        #otherwise start looking for the zip file
        else:
            if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip"):
                if not os.path.isfile(datestring + "-citibike-tripdata.zip"):
                    os.system("curl -O https://s3.amazonaws.com/tripdata/" + datestring + "-citibike-tripdata.zip")
                ###  To move it I use the os.system() functions to run bash commands with arguments
                os.system("mv " + datestring + "-citibike-tripdata.zip " + os.getenv("PUIDATA"))
            ### unzip the csv 
            os.system("unzip " + os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv")
            ## NOTE: old csv citibike data had a different name structure. 
            if '2014' in datestring:
                os.system("mv " + datestring[:4] + '-' +  datestring[4:] + 
                          "\ -\ Citi\ Bike\ trip\ data.csv " + datestring + "-citibike-tripdata.csv")
            os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA"))
    ### One final check:
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        print ("WARNING!!! something is wrong: the file is not there!")

    else:
        print ("file in place, you can continue")

In [10]:
datestring = '201707'
getCitiBikeCSV(datestring)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [7]:
df = pd.read_csv(os.getenv("PUIDATA") + '/201707-citibike-tripdata.csv')

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

<b>General concept to test: </b>
<br>Younger people are more likely to ride Citi Bikes late at night

<b>Null hypothesis:</b><br>
The proportion of riders 35+ to total riders for trips starting at midnight-5 am is higher or equal
to the proportion of riders 35+ to total riders for trips starting at 5 am - midnight

<b>H0:<b>
$\frac{old riders {\mathrm{night}}}{all riders {\mathrm{night}}} >= \frac{old riders{\mathrm{day}}}{old riders{\mathrm{day}}}$

<br>Ha:
$\frac{old riders{\mathrm{night}}}{all riders{\mathrm{night}}} < \frac{old riders{\mathrm{day}}}{old riders{\mathrm{day}}}$

<br>
Alpha: 5% <br>
Confidence level: 95%

In [ ]:
# Clean and add fields to dataframe
df['date'] = pd.to_datetime(df['starttime'])
df['age'] = 2017 - df['birth year']
df['hour'] = df['date'].dt.hour

In [ ]:
# Present sample of data frame
df.head()

In [ ]:
# Reduce df (simpler method than dropping long list of columns)
df = df[['hour', 'age']]

In [ ]:
# Clean up and create flag/dummy fields for binary data
df['age_group'] = df['age'] >= 35
age_dict = {True: '35+', False: '<35'}
df['age_group'] = df['age_group'].map(age_dict)

df['time'] = df['hour'] < 5
time_dict = {True: 'Late Night', False: 'Daytime'}
df['time'] = df['time'].map(time_dict)

In [ ]:
# Present sample of redacted dataframe
df.head()

In [ ]:
# Plot distribution of age groups by hour
fig = plt.figure(figsize=(12,8))

df2 = df[df['age_group'] == "35+"]
df2.groupby('hour').size().plot(kind='bar', color='blue', alpha=0.5)

df1 = df[df['age_group'] == "<35"]
df1.groupby('hour').size().plot(kind='bar', color='red', alpha=0.5, label='nine')
plt.xlabel('Hour of Day')
plt.ylabel('Count of Rides')
plt.title('Rides by Age Group by Hour\n' +
          'Red bars for <35 group; blue bars for 35+ group\n' + 
          '(pure blue for times with more 35 riders+; pure red for more <35 riders)', fontsize = 14)

In [ ]:
# Simple plot of group sizes
fig = plt.figure(figsize=(10,5))
plt.style.use('ggplot')
df.groupby(['time', 'age_group']).size().plot(kind='bar', color = 'Steelblue')
plt.xlabel('Category')
plt.ylabel('Rides')
plt.title('Citibike Rides by Age Group/Time Group')